# Todos
1. Create Empty HDF5 File
1. Read in Previous Orders
    - Split Between Train and Test
    - For Train Write out the Xs
    - For Test Write out the Xs
2. Read in the Dependent Variables
    - Split Between Train and Test
    - For Train Write out the order_month and the Ys
    - For Test Write out the order_month and the Ys

In [1]:
import numpy as np
import h5py
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 1000)
pd.set_option('max_columns',100)

# Create Empty HDF5 File

In [2]:
ord_hist_train = h5py.File('data/online_and_order/order_history_train.h5', 'w')
ord_hist_test = h5py.File('data/online_and_order/order_history_test.h5', 'w')

In [3]:
train_test_split_date = '2017-09-1'

# Save Previous Orders

In [4]:
previous_orders = pd.read_pickle('data/online_and_order/final_previous_order_df.pkl')

In [5]:
previous_orders = previous_orders.drop(
    ['ordno_current', 'months_before', 'days_before'],axis=1
)

#### Training

In [6]:
train_previous_orders = previous_orders[
    previous_orders['orderdate']<= pd.to_datetime(train_test_split_date)
].copy()

In [7]:
del(train_previous_orders['orderdate'])
del(train_previous_orders['ordno'])
#del(final_pd)

In [8]:
train_X_orders = train_previous_orders.values
train_X_orders = train_X_orders.reshape((-1,14, 253))

In [9]:
ord_hist_train.create_dataset('train_X_orders', data=train_X_orders)

<HDF5 dataset "train_X_orders": shape (86994, 14, 253), type "<f8">

#### Test

In [10]:
test_previous_orders = previous_orders[
    previous_orders['orderdate']> pd.to_datetime(train_test_split_date)
].copy()

In [11]:
del(test_previous_orders['orderdate'])
del(test_previous_orders['ordno'])

In [12]:
test_X_orders = test_previous_orders.values

test_X_orders = test_X_orders.reshape((-1,14, 253))

In [13]:
ord_hist_test.create_dataset('test_X_orders', data=test_X_orders)

<HDF5 dataset "test_X_orders": shape (15921, 14, 253), type "<f8">

# Save Session Data

In [14]:
previous_sessions = pd.read_pickle('data/online_and_order/previous_online_sessions_by_week.pkl')

In [15]:
previous_sessions = previous_sessions.drop(
    ['ordno', 'weeks_before', 'days_before'],axis=1
)

#### Training

In [16]:
train_previous_sessions = previous_sessions[
    previous_sessions['orderdate']<= pd.to_datetime(train_test_split_date)
].copy()

In [17]:
del(train_previous_sessions['orderdate'])

In [18]:
train_X_sessions = train_previous_sessions.values
train_X_sessions = train_X_sessions.reshape((-1,53, 24))

In [19]:
ord_hist_train.create_dataset('train_X_sessions', data=train_X_sessions)

<HDF5 dataset "train_X_sessions": shape (86994, 53, 24), type "<f8">

#### Test

In [20]:
test_previous_sessions = previous_sessions[
    previous_sessions['orderdate']> pd.to_datetime(train_test_split_date)
].copy()

In [21]:
del(test_previous_sessions['orderdate'])

In [22]:
test_X_sessions = test_previous_sessions.values
test_X_sessions = test_X_sessions.reshape((-1,53, 24))

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [23]:
ord_hist_test.create_dataset('test_X_sessions', data=test_X_sessions)

<HDF5 dataset "test_X_sessions": shape (15921, 53, 24), type "<f8">

# Save Dependent Variables

In [24]:
dependent_vars = pd.read_pickle('data/online_and_order/dependent_vars.pkl')

In [25]:
assert len(set(dependent_vars['ordno'])) == len(set(previous_orders['ordno']))

#### Convert Orders From Counts to BInary

In [26]:
dependent_vars[
    ['P1:1.0', 'P1:2.0', 'P1:3.0', 'P1:4.0', 'P1:5.0', 'P1:7.0']
] = (dependent_vars[
    ['P1:1.0', 'P1:2.0', 'P1:3.0', 'P1:4.0', 'P1:5.0', 'P1:7.0']
]>0).astype(int)

#### Training

In [27]:
dependent_vars.head()

,ordno,orderdate,ordermonth,P1:1.0,P1:2.0,P1:3.0,P1:4.0,P1:5.0,P1:7.0
0,1,2017-06-12 08:27:59,6,1,0,0,0,0,0
1,2,2016-03-29 13:04:27,3,1,0,0,0,0,0
2,3,2017-01-31 19:16:49,1,1,0,0,0,0,0
3,4,2017-08-14 21:22:10,8,1,0,0,0,0,0
4,5,2016-01-16 16:59:06,1,1,0,0,0,0,0


In [28]:
train_dependent_pd = dependent_vars[
    dependent_vars['orderdate']<= pd.to_datetime(train_test_split_date)
]

In [29]:
train_X_order_month = train_dependent_pd['ordermonth'].values.reshape((-1,1))

In [30]:
train_Y_order = train_dependent_pd[
    ['P1:1.0', 'P1:2.0', 'P1:3.0', 'P1:4.0', 'P1:5.0', 'P1:7.0']
].values

In [31]:
ord_hist_train.create_dataset('train_X_order_month', data=train_X_order_month)

<HDF5 dataset "train_X_order_month": shape (86994, 1), type "<i8">

In [32]:
ord_hist_train.create_dataset('train_Y_order', data=train_Y_order)

<HDF5 dataset "train_Y_order": shape (86994, 6), type "<i8">

#### Test

In [33]:
test_dependent_pd = dependent_vars[dependent_vars['orderdate']> pd.to_datetime(train_test_split_date)]

In [34]:
test_X_order_month = test_dependent_pd['ordermonth'].values.reshape((-1,1))

In [35]:
test_Y_order = test_dependent_pd[['P1:1.0', 'P1:2.0', 'P1:3.0', 'P1:4.0', 'P1:5.0', 'P1:7.0']].values

In [36]:
ord_hist_test.create_dataset('test_X_order_month', data=test_X_order_month)

<HDF5 dataset "test_X_order_month": shape (15921, 1), type "<i8">

In [37]:
ord_hist_test.create_dataset('test_Y_order', data=test_Y_order)

<HDF5 dataset "test_Y_order": shape (15921, 6), type "<i8">

# Close the HDF5 File

In [38]:
ord_hist_train.keys()

<KeysViewHDF5 ['train_X_order_month', 'train_X_orders', 'train_X_sessions', 'train_Y_order']>

In [39]:
ord_hist_test.keys()

<KeysViewHDF5 ['test_X_order_month', 'test_X_orders', 'test_X_sessions', 'test_Y_order']>

In [40]:
ord_hist_train.close()

In [41]:
ord_hist_test.close()